2 step

_Eq. (1) can be written as_
$$\frac{\partial u}{\partial t} + \frac{\partial F(u)}{\partial x} = 0$$ 
_where $F(u)=\frac{1}{2}u^2$._

In [1]:
import sys 
import os   
sys.path.append(os.getcwd()+'/..')
import numpy as np
import math
import matplotlib.pyplot as plt 
import nm_lib as nm
from typing import Type
from matplotlib import animation
import matplotlib
matplotlib.use('TkAgg')

# --- Values reused in other tasks of ex_3 ---
x0 = -1.4 
xf = 2.0
xc = 0.7
W = 0.1
pi = math.pi
CFL = 0.5
A = 0.02
# ---------------------------

def tfunc_Burgers(xx, A, B):
    return A * (math.tanh((xx + xc) / W) - math.tanh((xx - xc) / W)) + B

def discretisation(x0, xf, nump, dt_float: Type[float]):
    nint = nump - 1
    dh = dt_float((xf - x0) / nint)
    xx = dt_float(np.arange(nump) * dh + x0) 
    xx_half = dt_float(np.arange(nump) * dh + x0 + dh * 0.5)
    return nint, dh, xx, xx_half

def run_Burgers(nump, nt, CFL, A, B, ddx1 = lambda x,y: nm.deriv_cent(x,y), ddx2 = lambda x,y: nm.deriv_bck(x,y), bnd_limits: list=[1,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc_Burgers(xx[i], A, B)
 
    t1, unnt1 = nm.evolv_Lax_uadv_burgers(xx, hh, nt, CFL, ddx1, 'wrap', bnd_limits)
    t2, unnt2 = nm.evolv_uadv_burgers(xx, hh, nt, CFL, ddx2, 'wrap', bnd_limits)
    t3, unnt3 = nm.evolv_Rie_uadv_burgers(xx, hh, nt, CFL,  'wrap', bnd_limits)  

    plt.figure()
    plt.xlim(xx[0], xx[nump-1])
    plt.ylim(unnt1[:,0].min() - 0.1 * (unnt1[:,0].max() - unnt1[:,0].min()), unnt1[:,0].max() + 0.1 * (unnt1[:,0].max() - unnt1[:,0].min()))
    plt.xlabel("x")
    plt.ylabel("u")  
    line1, = plt.plot([], [], '--', label='Lax solution')
    line2, = plt.plot([], [], '--', label='Backward difference solution')
    line3, = plt.plot([], [], '--', label='Rusanov solution')

    plt.legend(loc='lower right') 
    time_text1 = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
    time_text2 = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
    time_text3 = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes) 
 
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        line3.set_data([], [])
        time_text1.set_text('')
        time_text2.set_text('')
        time_text3.set_text('')
        return line1, line2, line3
 
    def update(frame):
        line1.set_data(xx, unnt1[:,frame])  
        line2.set_data(xx, unnt2[:,frame]) 
        line3.set_data(xx, unnt3[:,frame])   
        time_text1.set_text("Time1 = "+str(t1[frame]))
        time_text2.set_text("Time2 = "+str(t2[frame]))
        time_text3.set_text("Time3 = "+str(t3[frame]))
        return line1, line2, line3
 
    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t1.size-1, blit=True)
    plt.show(block=True)

nump = 129   
nt = 301
B = 0.0
run_Burgers(nump, nt, CFL, A, B)  